In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sorted_alpha import sorted_alpha
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
from scipy import interpolate
from scipy.interpolate import interp1d
from frame_count import frame_count
from scipy.signal import savgol_filter
from pykalman import UnscentedKalmanFilter


In [ ]:
import torch
from strongsort import StrongSORT

tracker = StrongSORT(model_weights="osnet_x0_25_msmt17.pt", device=torch.device("cpu"), fp16=False)


In [ ]:
pose = '/Users/andrei-macpro/Documents/Data/pose/play_openpose/1047_play'
detections = sorted_alpha(pose)

In [ ]:
path_video = '/Users/andrei-macpro/Documents/Data/videos/play_videos/1047_play.mp4'
cap = cv2.VideoCapture(path_video)

In [ ]:
frames = [int(x.split("_")[1].split('.')[0]) for x in detections]

In [ ]:
images = []
for frame in frames[:10]:
    cap.set(1,frame)
    ret, image = cap.read()
    images.append(image)


In [ ]:
detections

In [ ]:
skeletons = []
for det in detections[:10]: 
    temp_list = []
    df = pd.read_csv(os.path.join(pose,det))
    columns = df.columns[1:]
    probs = [df.iloc[:,column+1].mean() for column in range(2, len(columns), 3)]
    detections_x = [df.iloc[:,column+1].min() for column in range(0, len(columns), 3)]                                   
    detections_y  = [df.iloc[:,column+1].min() for column in range(1, len(columns), 3)]
    width = [df.iloc[:,column+1].max() for column in range(0, len(columns), 3)]
    height = [df.iloc[:,column+1].max() for column in range(1, len(columns), 3)]
    k=1
    for x,y,w,h,p in zip(detections_x, detections_y, width, height, probs):
        temp_list.append([x,y,w-x,h-y, round(p, 2), 'person '+str(k)])
        k+=1
    skeletons.append(np.array(temp_list))
             

In [ ]:
type(skeletons[1][0][0]), skeletons[1][0][0]

# probs is the mean probability lol 

In [ ]:
sk_1 = np.array(skeletons[1])
# convert sk_1 from 6 rows and 1 column to 1 row and 6 columns
sk_1 = sk_1.reshape(1, -1)
sk_1  





In [ ]:
skeletons[1].shape

In [ ]:
sk_1[:, :4]

In [ ]:
# convert skeletons[0][0] into a numpy array
sk_1 = np.array(skeletons[0][0])
sk_1

In [ ]:
for detection, frame in zip(skeletons, images):
    print(detection, frame.shape)
    tracks = tracker.update(detection, frame)

In [ ]:
def xyxy2xywh(x):
    # Convert nx4 boxes from [x1, y1, x2, y2] to [x, y, w, h] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = (x[:, 0] + x[:, 2]) / 2  # x center
    y[:, 1] = (x[:, 1] + x[:, 3]) / 2  # y center
    y[:, 2] = x[:, 2] - x[:, 0]  # width
    y[:, 3] = x[:, 3] - x[:, 1]  # height
    return y


xyxys

In [ ]:
dets = skeletons[1]
xyxys = dets[:, :4]
xyxys = dets[:, 0:4]
confs = dets[:, 4]
clss = dets[:, 5]

In [ ]:
def _get_features(self, bbox_xywh, ori_img): # where can i find the bbox_xywh?
    im_crops = []
    for box in bbox_xywh:
        x1, y1, x2, y2 = self._xywh_to_xyxy(box)
        im = ori_img[y1:y2, x1:x2]
        im_crops.append(im)
    if im_crops:
        features = self.model(im_crops)
    else:
        features = np.array([])
    return features

In [ ]:
_get_features(self, bbox_xywh, ori_img) # what is bbox_xywh



In [ ]:
 bbox_tlwh = self._xywh_to_tlwh(xywhs)
xywhs = xyxy2xywh(np.array(xyxys.astype(float)))

In [ ]:
xywhs = xyxy2xywh(np.array(xyxys.astype(float)))


In [ ]:
xywhs